In [ ]:
# default_exp models.RNNPlus

# RNNPlus

> These are RNN, LSTM and GRU PyTorch implementations created by Ignacio Oguiza - oguiza@gmail.com based on:

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.core import *
from tsai.models.layers import *

In [ ]:
a = !python  -V
p = a[0].split(' ')
print(f'python         : {p[1]}')
print('fastai         :', fastai.__version__)
print('fastcore       :', fastcore.__version__)
print('torch          :', torch.__version__)
print('scipy          :', sp.__version__)
print('numpy          :', np.__version__)
print('pandas         :', pd.__version__)
print('matplotlib     :', matplotlib.__version__)
print(f'Total RAM      : {bytes2GB(psutil.virtual_memory().total):6.2f} GB')
print(f'Available RAM  : {bytes2GB(psutil.virtual_memory().available):6.2f} GB')
print(f'Used RAM       : {bytes2GB(psutil.virtual_memory().used):6.2f} GB')
print('n_cpus         :', cpus)
iscuda = torch.cuda.is_available()
if iscuda: print('device         : {} ({})'.format(device, torch.cuda.get_device_name(0)))
else: print('device         :', device)

python         : 3.7.6
fastai         : 2.0.18
fastcore       : 1.2.5
torch          : 1.6.0
scipy          : 1.5.2
numpy          : 1.19.1
pandas         : 1.1.3
matplotlib     : 3.3.2
Total RAM      :   8.00 GB
Available RAM  :   1.65 GB
Used RAM       :   3.23 GB
n_cpus         : 8
device         : cpu


In [ ]:
#hide
# example
# rnn = nn.LSTM(10, 20, 2)
# input = torch.randn(5, 3, 10)
# h0 = torch.randn(2, 3, 20)
# c0 = torch.randn(2, 3, 20)
# output, (hn, cn) = rnn(input, (h0, c0))

In [ ]:
#export
class _RNNPlus_Base(Module):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, n_layers=1, bias=True, rnn_dropout=0, bidirectional=False, fc_dropout=0., 
                 last_step=True, flatten=False, custom_head=None, y_range=None, **kwargs):
        if flatten: assert seq_len, 'you need to enter a seq_len to use flatten=True'
        self.ls, self.fl = last_step, flatten
        self.rnn = self._cell(c_in, hidden_size, num_layers=n_layers, bias=bias, batch_first=True, dropout=rnn_dropout, bidirectional=bidirectional)
        if last_step: self.last_step = LastStep()
        if flatten: self.flatten = Reshape(-1)
        
        # Head
        self.head_nf = seq_len * hidden_size * (1 + bidirectional) if flatten and not last_step else hidden_size * (1 + bidirectional)
        self.c_out = c_out
        if custom_head: self.head = custom_head(self.head_nf, c_out) # custom head must have all required kwargs
        else: self.head = self.create_head(self.head_nf, c_out, fc_dropout=fc_dropout, y_range=y_range)

    def forward(self, x): 
        x = x.transpose(2,1) # [batch_size x n_vars x seq_len] --> [batch_size x seq_len x n_vars]
        output, _ = self.rnn(x) # output from all sequence steps: [batch_size x seq_len x hidden_size * (1 + bidirectional)]
        if self.ls: output = self.last_step(output) #             [batch_size x hidden_size * (1 + bidirectional)]
        if self.fl: output = self.flatten(output) #               [batch_size x seq_len * hidden_size * (1 + bidirectional)]
        if not self.ls and not self.fl: output = output.transpose(2,1)
        return self.head(output)
    
    def create_head(self, nf, c_out, fc_dropout=0., y_range=None):
        layers = [nn.Dropout(fc_dropout)] if fc_dropout else []
        layers += [nn.Linear(self.head_nf, c_out)]
        if y_range is not None: layers += [SigmoidRange(*y_range)]
        return nn.Sequential(*layers)

    
class RNNPlus(_RNNPlus_Base):
    _cell = nn.RNN
    
class LSTMPlus(_RNNPlus_Base):
    _cell = nn.LSTM
    
class GRUPlus(_RNNPlus_Base):
    _cell = nn.GRU

In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
test_eq(RNNPlus(c_in, c_out, seq_len, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5, flatten=True)(xb).shape, 
        [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5, flatten=True)(xb).shape, 
        [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(GRUPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])

In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
custom_head = partial(create_mlp_head, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, flatten=True, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_pool_head, concat_pool=True, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_pool_plus_head, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_conv_head)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])

In [ ]:
from tsai.data.all import *
dsid = 'NATOPS' 
bs = 16
X, y, splits = get_UCR_data(dsid, return_split=False)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=bs, num_workers=0, shuffle=False)

In [ ]:
model = LSTMPlus(dls.vars, dls.c, dls.len)
print(model.head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): Linear(in_features=100, out_features=6, bias=True)
)


epoch,train_loss,valid_loss,accuracy,time
0,1.801460,1.773930,0.205556,00:04


In [ ]:
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=True)
print(model.head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): Linear(in_features=5100, out_features=6, bias=True)
)


epoch,train_loss,valid_loss,accuracy,time
0,1.228263,0.721918,0.711111,00:02


In [ ]:
custom_head = partial(create_pool_head, concat_pool=True)
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=False, custom_head=custom_head)
print(model.head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): GACP1d(
    (gacp): AdaptiveConcatPool1d(
      (ap): AdaptiveAvgPool1d(output_size=1)
      (mp): AdaptiveMaxPool1d(output_size=1)
    )
    (flatten): Flatten(full=False)
  )
  (1): LinBnDrop(
    (0): Linear(in_features=200, out_features=6, bias=True)
  )
)


epoch,train_loss,valid_loss,accuracy,time
0,1.731815,1.636969,0.494444,00:05


In [ ]:
custom_head = partial(create_pool_plus_head, concat_pool=True)
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=False, custom_head=custom_head)
print(model.head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): AdaptiveConcatPool1d(
    (ap): AdaptiveAvgPool1d(output_size=1)
    (mp): AdaptiveMaxPool1d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=200, out_features=512, bias=False)
  (4): ReLU(inplace=True)
  (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=512, out_features=6, bias=False)
)


epoch,train_loss,valid_loss,accuracy,time
0,0.886501,1.516735,0.477778,00:05


In [ ]:
m = RNNPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

RNNPlus(
  (rnn): RNN(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (last_step): LastStep()
  (flatten): Reshape(bs, -1)
  (head): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=200, out_features=2, bias=True)
  )
)
(81802, True)


torch.Size([16, 2])

In [ ]:
m = LSTMPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

LSTMPlus(
  (rnn): LSTM(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (last_step): LastStep()
  (flatten): Reshape(bs, -1)
  (head): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=200, out_features=2, bias=True)
  )
)
(326002, True)


torch.Size([16, 2])

In [ ]:
m = GRUPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

GRUPlus(
  (rnn): GRU(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (last_step): LastStep()
  (flatten): Reshape(bs, -1)
  (head): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=200, out_features=2, bias=True)
  )
)
(244602, True)


torch.Size([16, 2])

In [ ]:
#hide
out = create_scripts()
beep(out)